# Assignment 1

student: Thomas Reolon

repository: [github](https://github.com/thomasreolon/NLU_unitn)


## Assignment: Working with Dependency Graphs (Parses)

The objective of the assignment is to learn how to work with dependency graphs by defining functions.

Read [spaCy documentation on dependency parser](https://spacy.io/api/dependencyparser) to learn provided methods.

Define functions to:
- extract a path of dependency relations from the ROOT to a token
- extract subtree of a dependents given a token
- check if a given list of tokens (segment of a sentence) forms a subtree
- identify head of a span, given its tokens
- extract sentence subject, direct object and indirect object spans

**Setting up environment**

In [1]:
!python -m spacy download en_core_web_md | cat > hide_cell_output.txt

In [2]:
import spacy
import en_core_web_md

nlp = en_core_web_md.load()
#parser = nlp.get_pipe("parser") # component of nlp that create dep_graph

##### **1. extract a path of dependency relations from the ROOT to a token**

> get_path_from_root



In [3]:
def get_path_from_root(sentence:str):
  """returns a dict {token:list_of_dependencies} for every torken in the sentence"""
  doc = nlp(sentence)
  paths = {}
  for token in doc:
    tmp = token
    path = []
    while tmp != tmp.head:              # while we are not at root
      path.append(tmp.dep_)             # append dependency to path (a dependency is an arc in the graph)
      tmp = tmp.head                    # move to parent node
    path.append('ROOT')
    paths[token] = list(reversed(path)) # path starts from ROOT
  return paths

In [4]:
# small test

sentences = [
  'Mary was having breakfast two minutes ago.',
  'You can watch the movie while cooking',
  'I saw a man with a telescope.'
]


for sent in sentences:
  print('\n----------', sent, '----------')
  all_paths = get_path_from_root(sent)        # process sentence
  for tok, path in all_paths.items():
    print(f'token:{tok.text+" "*(10-len(tok))}: {path}')


---------- Mary was having breakfast two minutes ago. ----------
token:Mary      : ['ROOT', 'nsubj']
token:was       : ['ROOT', 'aux']
token:having    : ['ROOT']
token:breakfast : ['ROOT', 'dobj']
token:two       : ['ROOT', 'advmod', 'npadvmod', 'nummod']
token:minutes   : ['ROOT', 'advmod', 'npadvmod']
token:ago       : ['ROOT', 'advmod']
token:.         : ['ROOT', 'punct']

---------- You can watch the movie while cooking ----------
token:You       : ['ROOT', 'nsubj']
token:can       : ['ROOT', 'aux']
token:watch     : ['ROOT']
token:the       : ['ROOT', 'dobj', 'det']
token:movie     : ['ROOT', 'dobj']
token:while     : ['ROOT', 'advcl', 'mark']
token:cooking   : ['ROOT', 'advcl']

---------- I saw a man with a telescope. ----------
token:I         : ['ROOT', 'nsubj']
token:saw       : ['ROOT']
token:a         : ['ROOT', 'dobj', 'det']
token:man       : ['ROOT', 'dobj']
token:with      : ['ROOT', 'dobj', 'prep']
token:a         : ['ROOT', 'dobj', 'prep', 'pobj', 'det']
token:telesc

##### **2. extract subtree of a dependents given a token**

> get_subtree

In [5]:
def get_subtree(sentence:str):
  """returns a dict {token:subtree(token)} for every token in the sentence"""
  doc = nlp(sentence)
  subtrees = {}
  for token in doc:
    subtrees[token] = [t.text for t in token.subtree] # iterate through subtree generator and extract tokens
  return subtrees

In [6]:
# small test

for sent in sentences:
  print('\n----------', sent, '----------')
  subtrees = get_subtree(sent)
  for tok, st in subtrees.items():
    print(f'token:{tok.text+" "*(10-len(tok))} subtree: {st}')



---------- Mary was having breakfast two minutes ago. ----------
token:Mary       subtree: ['Mary']
token:was        subtree: ['was']
token:having     subtree: ['Mary', 'was', 'having', 'breakfast', 'two', 'minutes', 'ago', '.']
token:breakfast  subtree: ['breakfast']
token:two        subtree: ['two']
token:minutes    subtree: ['two', 'minutes']
token:ago        subtree: ['two', 'minutes', 'ago']
token:.          subtree: ['.']

---------- You can watch the movie while cooking ----------
token:You        subtree: ['You']
token:can        subtree: ['can']
token:watch      subtree: ['You', 'can', 'watch', 'the', 'movie', 'while', 'cooking']
token:the        subtree: ['the']
token:movie      subtree: ['the', 'movie']
token:while      subtree: ['while']
token:cooking    subtree: ['while', 'cooking']

---------- I saw a man with a telescope. ----------
token:I          subtree: ['I']
token:saw        subtree: ['I', 'saw', 'a', 'man', 'with', 'a', 'telescope', '.']
token:a          subtree:

##### **3. check if a given list of tokens (segment of a sentence) forms a subtree**

> is_subtree

In [7]:
def is_subtree(sentence:str, list_of_tokens:list):
  """check if a list of tokens is a subtree"""
  if len(list_of_tokens)==0: return True                            # default answer for empty graph (True)
  input_subtree = [str(t) for t in list_of_tokens]                  # in case list of token was a Span instance or a Doc
  possible_subtrees = [s for _,s in get_subtree(sentence).items()]  # get all possible subtree using function 2
  return input_subtree in possible_subtrees                         # if list_of_tokens is a valid subtree it must be in the possible subtrees

In [8]:
# small test

doc = nlp(sent)
print(is_subtree('I saw a man with a telescope.', ['with', 'a', 'telescope']))
print(is_subtree('I saw a man with a telescope.', ['with', 'saw', 'telescope']))

print(is_subtree(  sentences[0], nlp(sentences[0])[-4:-1]  ))  # Mary was having breakfast two minutes ago.   [ two, minutes, ago ]
print(is_subtree(  sentences[0], nlp(sentences[0])[-4:]  ))    # Mary was having breakfast two minutes ago.   [ two, minutes, ago, . ]

True
False
True
False


##### **4. identify head of a span, given its tokens**

> get_head_of_span

In [9]:
def get_head_of_span(span):
  """return the head node of a span, as a Token object, can handle multiple inputs"""
  if isinstance(span, spacy.tokens.Span):     # we can extract the root of a span with the 'root' attribute
    return span.root
  elif isinstance(span, spacy.tokens.Doc):    # if we have a doc, doc[:] returns a span of the whole sentence
    return span[:].root
  elif isinstance(span, list):                # if we have a list of tokens (eg. ['i', 'like', 'pizza']) we can convert it into a string and process it using spacy
    return nlp(' '.join(span))[:].root
  elif isinstance(span, str):                 # if we have a string we can obtain a Doc using nlp, then get the span with [:], then return root
    return nlp(span)[:].root

In [10]:
# small test

span = sentences[0].split()[-4:]
print(span, '---->', get_head_of_span(span))

span = nlp(sentences[1])[:4]
print(span, '---->', get_head_of_span(span))
  

['breakfast', 'two', 'minutes', 'ago.'] ----> breakfast
You can watch the ----> watch


##### **5. extract sentence subject, direct object and indirect object spans**

> extract_sub_ind_dir

In [11]:
def extract_sub_ind_dir(sentence:str):
  """returns subject, indirect obj, direct obj of a sentence

  Arguments
  ---------
  sentence:          str
    the sentence to process, eg. 'I saw Luca.'

  returns a dict of lists
  """
  out = {'SUBJECT':[], 'DIRECT-OBJECT':[], 'INDIRECT-OBJECT':[]}
  subjs = {'nsubjpass', 'csubj', 'expl', 'csubjpass'}
  doc = nlp(sentence)
  for token in doc:
    if   token.dep_=='dative' : out['INDIRECT-OBJECT'] = [str(t) for t in token.subtree]
    elif token.dep_=='dobj':    out['DIRECT-OBJECT'] = [str(t) for t in token.subtree]
    elif token.dep_ in subjs:   out['SUBJECT'] = [str(t) for t in token.subtree]
    elif token.dep_=='nsubj' and len(out['SUBJECT'])==0:
      # sometimes there is a token with a nsubj dependency and another one ( eg. csubj )
      # we give priority to the others over nsubj
      # eg. What you did was unforgivable --> nsubj=you,  csubj=what you did 
      out['SUBJECT'] = [str(t) for t in token.subtree]  

  return out

In [12]:
# small test

# 'her' is an indirect object, which, in spaCy, is identified by 'dative'  ((even if stack overflow said 'iobj'))
s = 'Please give her a copy of the article by tomorrow morning'
print(s, '->', extract_sub_ind_dir(s))

print('')
for sent in sentences:
  print(sent, '-->', extract_sub_ind_dir(sent))

Please give her a copy of the article by tomorrow morning -> {'SUBJECT': [], 'DIRECT-OBJECT': ['a', 'copy', 'of', 'the', 'article'], 'INDIRECT-OBJECT': ['her']}

Mary was having breakfast two minutes ago. --> {'SUBJECT': ['Mary'], 'DIRECT-OBJECT': ['breakfast'], 'INDIRECT-OBJECT': []}
You can watch the movie while cooking --> {'SUBJECT': ['You'], 'DIRECT-OBJECT': ['the', 'movie'], 'INDIRECT-OBJECT': []}
I saw a man with a telescope. --> {'SUBJECT': ['I'], 'DIRECT-OBJECT': ['a', 'man', 'with', 'a', 'telescope'], 'INDIRECT-OBJECT': []}


In [13]:
# test subjects
sents = [
  'I like pizza.',  #  I
  'Mary is cutting a carrot',  #  Mary
  'A carrot is being cut by Mary.',  #  A carrot
  'Obviously, the new design has a greater user interface',  #  the new design  (expl: Obviously)
  'What you did was unforgivable',  # What you did
  'What you did was not appreciated by the others',   # What you did
  'There is only one car in the park'  # There
]

for sent in sents:
  print(sent, '-->', extract_sub_ind_dir(sent))

I like pizza. --> {'SUBJECT': ['I'], 'DIRECT-OBJECT': ['pizza'], 'INDIRECT-OBJECT': []}
Mary is cutting a carrot --> {'SUBJECT': ['Mary'], 'DIRECT-OBJECT': ['a', 'carrot'], 'INDIRECT-OBJECT': []}
A carrot is being cut by Mary. --> {'SUBJECT': ['A', 'carrot'], 'DIRECT-OBJECT': [], 'INDIRECT-OBJECT': []}
Obviously, the new design has a greater user interface --> {'SUBJECT': ['the', 'new', 'design'], 'DIRECT-OBJECT': ['a', 'greater', 'user', 'interface'], 'INDIRECT-OBJECT': []}
What you did was unforgivable --> {'SUBJECT': ['What', 'you', 'did'], 'DIRECT-OBJECT': ['What'], 'INDIRECT-OBJECT': []}
What you did was not appreciated by the others --> {'SUBJECT': ['What', 'you', 'did'], 'DIRECT-OBJECT': ['What'], 'INDIRECT-OBJECT': []}
There is only one car in the park --> {'SUBJECT': ['There'], 'DIRECT-OBJECT': [], 'INDIRECT-OBJECT': []}


## Assignment: Training Transition-Based Dependency Parser (Optional & Advanced)

- Modify [NLTK Transition parser](https://github.com/nltk/nltk/blob/develop/nltk/parse/transitionparser.py)'s `Configuration` class to use better features.
- Evaluate the features comparing performance to the original
- Replace `SVM` classifier with an alternative of your choice.

In [14]:
from nltk.parse.transitionparser import *  # contains Configuration, TransitionParser and more

In [15]:
# from: https://github.com/nltk/nltk/blob/develop/nltk/parse/transitionparser.py
# with some changes to:
#  --> config: 
#         removed lemma:    noticeable impvrovment (probably redundant infos with 'word' argument)
#         word.lower():     did not impove accuracy, but i kept it
#         add dependencies: little improvement
#
#  --> model: SVM classifier kernel change from POLINOMIAL to RBF (and some hyperparameters)
#         incrementing C:   noticeable improvement (probably the boundary feature space is not regular)
#         decreasing gamma: little improvement (remove this change if you are training on the whole treebank dataset (if you have many training points you can have an higher gamma))

class MyConfiguration(Configuration):
  def __init__(self, dep_graph):
    super().__init__(dep_graph)

  def extract_features(self):
    result = []
    if len(self.stack) > 0:
      # Stack 0
      stack_idx0 = self.stack[len(self.stack) - 1]
      token = self._tokens[stack_idx0]
      if self._check_informative(token["word"], True):
        result.append("STK_0_FORM_" + token["word"].lower())
      if self._check_informative(token["tag"]):
        result.append("STK_0_POS_" + token["tag"])
      if "feats" in token and self._check_informative(token["feats"]):
        feats = token["feats"].split("|")
        for feat in feats:
          result.append("STK_0_FEATS_" + feat)
      # Stack 1
      if len(self.stack) > 1:
        stack_idx1 = self.stack[len(self.stack) - 2]
        token = self._tokens[stack_idx1]
        if self._check_informative(token["tag"]):
          result.append("STK_1_POS_" + token["tag"])

      # Left most, right most dependency of stack[0]
      left_most = 1000000
      right_most = -1
      dep_left_most = ""
      dep_right_most = ""
      for (wi, r, wj) in self.arcs:
        if wi == stack_idx0:
          if (wj > wi) and (wj > right_most):
            right_most = wj
            dep_right_most = r
          if (wj < wi) and (wj < left_most):
            left_most = wj
            dep_left_most = r
      if self._check_informative(dep_left_most):
        result.append("STK_0_LDEP_" + dep_left_most)
      if self._check_informative(dep_right_most):
        result.append("STK_0_RDEP_" + dep_right_most)

    # Check Buffered 0
    if len(self.buffer) > 0:
      # Buffer 0
      buffer_idx0 = self.buffer[0]
      token = self._tokens[buffer_idx0]
      if self._check_informative(token["word"], True):
          result.append("BUF_0_FORM_" + token["word"].lower())
      if self._check_informative(token["tag"]):
          result.append("BUF_0_POS_" + token["tag"])
      if "feats" in token and self._check_informative(token["feats"]):
          feats = token["feats"].split("|")
          for feat in feats:
              result.append("BUF_0_FEATS_" + feat)
      if "deps" in token and len(token["deps"])>0:
        for d in token["deps"]:
          result.append("DEP_0_"+str(d))
      #if "head" in token and token['head']:
      #  result.append("HEAD_0_"+str(token['head']))

      
      # Buffer 1
      if len(self.buffer) > 1:
        buffer_idx1 = self.buffer[1]
        token = self._tokens[buffer_idx1]
        if self._check_informative(token["word"], True):
          result.append("BUF_1_FORM_" + token["word"])
        if self._check_informative(token["tag"]):
          result.append("BUF_1_POS_" + token["tag"])
        if "deps" in token and len(token["deps"])>0:
          for d in token["deps"]:
            result.append("DEP_1_"+str(d))
      if len(self.buffer) > 2:
        buffer_idx2 = self.buffer[2]
        token = self._tokens[buffer_idx2]
        if self._check_informative(token["tag"]):
          result.append("BUF_2_POS_" + token["tag"])
      if len(self.buffer) > 3:
        buffer_idx3 = self.buffer[3]
        token = self._tokens[buffer_idx3]
        if self._check_informative(token["tag"]):
          result.append("BUF_3_POS_" + token["tag"])
          # Left most, right most dependency of stack[0]
      left_most = 1000000
      right_most = -1
      dep_left_most = ""
      dep_right_most = ""
      for (wi, r, wj) in self.arcs:
        if wi == buffer_idx0:
          if (wj > wi) and (wj > right_most):
            right_most = wj
            dep_right_most = r
          if (wj < wi) and (wj < left_most):
            left_most = wj
            dep_left_most = r
      if self._check_informative(dep_left_most):
        result.append("BUF_0_LDEP_" + dep_left_most)
      if self._check_informative(dep_right_most):
        result.append("BUF_0_RDEP_" + dep_right_most)

    return result

In [16]:
class MyParser(TransitionParser):
  def __init__(self):
    super().__init__("arc-eager")

  def _create_training_examples_arc_eager(self, depgraphs, input_file):
    operation = Transition(self.ARC_EAGER)
    countProj = 0
    training_seq = []

    for depgraph in depgraphs:
      if not self._is_projective(depgraph):
        continue

      countProj += 1
      conf = MyConfiguration(depgraph)
      while len(conf.buffer) > 0:
        b0 = conf.buffer[0]
        features = conf.extract_features()
        binary_features = self._convert_to_binary_features(features)

        if len(conf.stack) > 0:
          s0 = conf.stack[len(conf.stack) - 1]
          # Left-arc operation
          rel = self._get_dep_relation(b0, s0, depgraph)
          if rel is not None:
            key = Transition.LEFT_ARC + ":" + rel
            self._write_to_file(key, binary_features, input_file)
            operation.left_arc(conf, rel)
            training_seq.append(key)
            continue

          # Right-arc operation
          rel = self._get_dep_relation(s0, b0, depgraph)
          if rel is not None:
            key = Transition.RIGHT_ARC + ":" + rel
            self._write_to_file(key, binary_features, input_file)
            operation.right_arc(conf, rel)
            training_seq.append(key)
            continue

          # reduce operation
          flag = False
          for k in range(s0):
            if self._get_dep_relation(k, b0, depgraph) is not None:
              flag = True
            if self._get_dep_relation(b0, k, depgraph) is not None:
              flag = True
          if flag:
            key = Transition.REDUCE
            self._write_to_file(key, binary_features, input_file)
            operation.reduce(conf)
            training_seq.append(key)
            continue

        # Shift operation as the default
        key = Transition.SHIFT
        self._write_to_file(key, binary_features, input_file)
        operation.shift(conf)
        training_seq.append(key)

    return training_seq

  def train(self, depgraphs, modelfile, verbose=True):
    try:
      input_file = tempfile.NamedTemporaryFile(
        prefix="transition_parse.train", dir=tempfile.gettempdir(), delete=False
      )

      if self._algorithm == self.ARC_STANDARD:
        self._create_training_examples_arc_std(depgraphs, input_file)
      else:
        self._create_training_examples_arc_eager(depgraphs, input_file)

      input_file.close()
      x_train, y_train = load_svmlight_file(input_file.name)
      model = svm.SVC(
          kernel="rbf",
          coef0=0,
          gamma=0.08, # ---> low = higher variance (easier similarity)
          C=4,        # ---> high = high penalization for errors
          verbose=verbose,
          probability=True,
      )

      model.fit(x_train, y_train)
      pickle.dump(model, open(modelfile, "wb"))
    finally:
      remove(input_file.name)
  
  def parse(self, depgraphs, modelFile):
    result = []
    # First load the model
    model = pickle.load(open(modelFile, "rb"))
    operation = Transition(self._algorithm)

    for depgraph in depgraphs:
      conf = MyConfiguration(depgraph)
      while len(conf.buffer) > 0:
        features = conf.extract_features()
        col = []
        row = []
        data = []
        for feature in features:
          if feature in self._dictionary:
            col.append(self._dictionary[feature])
            row.append(0)
            data.append(1.0)
        np_col = array(sorted(col))  # NB : index must be sorted
        np_row = array(row)
        np_data = array(data)

        x_test = sparse.csr_matrix(
            (np_data, (np_row, np_col)), shape=(1, len(self._dictionary))
        )

        prob_dict = {}
        pred_prob = model.predict_proba(x_test)[0]
        for i in range(len(pred_prob)):
          prob_dict[i] = pred_prob[i]
        sorted_Prob = sorted(prob_dict.items(), key=itemgetter(1), reverse=True)

        # Note that SHIFT is always a valid operation
        for (y_pred_idx, confidence) in sorted_Prob:
          y_pred = model.classes_[y_pred_idx]

          if y_pred in self._match_transition:
            strTransition = self._match_transition[y_pred]
            baseTransition = strTransition.split(":")[0]

            if baseTransition == Transition.LEFT_ARC:
              if (
                  operation.left_arc(conf, strTransition.split(":")[1])
                  != -1
              ):break
            elif baseTransition == Transition.RIGHT_ARC:
              if (
                  operation.right_arc(conf, strTransition.split(":")[1])
                  != -1
              ):break
            elif baseTransition == Transition.REDUCE:
              if operation.reduce(conf) != -1:break
            elif baseTransition == Transition.SHIFT:
              if operation.shift(conf) != -1:break
          else:
            raise ValueError(
                "The predicted transition is not recognized, expected errors"
            )

        # Finish with operations build the dependency graph from Conf.arcs

        new_depgraph = deepcopy(depgraph)
        for key in new_depgraph.nodes:
            node = new_depgraph.nodes[key]
            node["rel"] = ""
            # With the default, all the token depend on the Root
            node["head"] = 0
        for (head, rel, child) in conf.arcs:
            c_node = new_depgraph.nodes[child]
            c_node["head"] = head
            c_node["rel"] = rel
      result.append(new_depgraph)

    return result

In [17]:
import nltk
from nltk.parse import DependencyEvaluator
nltk.download('dependency_treebank')
from nltk.corpus import dependency_treebank

[nltk_data] Downloading package dependency_treebank to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/dependency_treebank.zip.


In [18]:
# default NLTK parser, arc-standard

train_size = 200
test_size  = 50

tp = TransitionParser('arc-standard')
tp.train(dependency_treebank.parsed_sents()[:train_size], 'tp.model')
parses = tp.parse(dependency_treebank.parsed_sents()[-test_size:], 'tp.model')

de = DependencyEvaluator(parses, dependency_treebank.parsed_sents()[-test_size:])
print(f'\n\nscores of basic Transition Parser from nltk = {de.eval()}')

 Number of training examples : 200
 Number of valid (projective) examples : 200
[LibSVM]

scores of basic Transition Parser from nltk = (0.8152709359605911, 0.8152709359605911)


In [19]:
# my random parser, arc-eager

# train
tp = MyParser()
tp.train(dependency_treebank.parsed_sents()[:train_size], 'tp2.model')

# test
input = dependency_treebank.parsed_sents()[-test_size:]
parses = tp.parse(input, 'tp2.model')
print(len(input), len(parses))
de = DependencyEvaluator(parses, input)
print(f'\n\nscores of MyParser = {de.eval()}')

[LibSVM]50 50


scores of MyParser = (0.8555008210180624, 0.8555008210180624)
